## Customized training

In [1]:
from src.models.model import *
from src.datasets.mydataset import *

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [2]:
from classy_vision.dataset import build_dataset
from classy_vision.generic.util import load_json
from classy_vision.meters import build_meters, AccuracyMeter, VideoAccuracyMeter
from classy_vision.tasks import ClassificationTask
from classy_vision.optim import build_optimizer
from classy_vision.losses import build_loss
from classy_vision.models import build_model

In [3]:
import json
config = load_json('configs/resnet50.json')

In [4]:
config

{'name': 'classification_task',
 'num_epochs': 90,
 'loss': {'name': 'CrossEntropyLoss'},
 'dataset': {'train': {'name': 'web_dataset',
   'batchsize_per_replica': 256,
   'num_samples': None,
   'shuffle': True,
   'img_folder': '/home/l/liny/ruofan/detectron2-1_website/datasets/train_imgs',
   'annot_path': '/home/l/liny/ruofan/detectron2-1_website/datasets/train_coords.txt'},
  'test': {'name': 'web_dataset',
   'batchsize_per_replica': 256,
   'num_samples': None,
   'shuffle': False,
   'img_folder': '/home/l/liny/ruofan/detectron2-1_website/datasets/val_imgs',
   'annot_path': '/home/l/liny/ruofan/detectron2-1_website/datasets/val_coords.txt'}},
 'meters': {'accuracy': {'topk': [1]}},
 'model': {'name': 'my_model',
  'num_classes': 2,
  'grid_num': 10,
  'input_ch_size': 9},
 'optimizer': {'name': 'sgd',
  'param_schedulers': {'lr': {'name': 'step', 'values': [0.1, 0.01, 0.001]}},
  'weight_decay': 0.0001,
  'momentum': 0.9}}

In [5]:
# Build dataset
datasets = {}
datasets["train"] = build_dataset(config["dataset"]["train"])
datasets["test"] = build_dataset(config["dataset"]["test"])

In [6]:
# Build model
model = build_model(config["model"])

In [7]:
# Build evaluation metric
meters = build_meters(config["meters"])

In [8]:
# Build loss, optimizer
loss = build_loss(config["loss"])
num_epochs = config["num_epochs"]
optimizer = build_optimizer(config["optimizer"])

task = (
    ClassificationTask()
    .set_loss(loss)
    .set_model(model)
    .set_optimizer(optimizer)
    .set_meters(meters)
    .set_num_epochs(num_epochs)
) 

for phase in ["train", "test"]:
    task.set_dataset(datasets[phase], phase)

In [ ]:
import time
import os

from classy_vision.trainer import LocalTrainer
from classy_vision.hooks import CheckpointHook
from classy_vision.hooks import LossLrMeterLoggingHook

hooks = [LossLrMeterLoggingHook(log_freq=4)]

checkpoint_dir = f"./output/classy_checkpoint_{time.time()}"
os.mkdir(checkpoint_dir)
hooks.append(CheckpointHook(checkpoint_dir, input_args={}))

task = task.set_hooks(hooks)

trainer = LocalTrainer()
trainer.train(task)

In [ ]:
import torch
x = torch.randn((1, 9, 10, 10))
with torch.no_grad():
    y_hat = model(x)

y_hat